# Essential Libraries

In [23]:
%load_ext autoreload

%autoreload 2

import configparser
import os
import time
import random
import pandas as pd
import requests
import openpyxl

from io import StringIO
from pathlib import Path
from bs4 import BeautifulSoup
from alive_progress import alive_bar

# some stuff I set up in a config file so I don't have to keep updating certain
# variables in every script
config = configparser.ConfigParser()
config.read('../src/config.ini')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['../src/config.ini']

# Starting Point

In [24]:
config = configparser.ConfigParser()
config.read('../src/config.ini')

['../src/config.ini']

In [25]:
# the output path is specified in the config.ini file
output = Path(config['paths']['output'])
# I want data for the 2022 through 2024 season
yearly_directories = [Path(output/f"mls_{year}") for year in range(2022,2025)]

In [26]:
# create output directory and sub-directories if doesnt exist
for directory in yearly_directories+[output]:
    try:
        assert directory.exists()
    except:
        os.mkdir(directory)

In [7]:
# I will be web-scraping alot, so I made this function as a result

def get_html_data(url, parser='html.parser') -> bs4.BeautifulSoup:
    '''
    Extract html data from specified url and return a bs4 object.
    Parser can be specified if needed. Default is html.parser.
    '''
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, parser)
    
    return soup

In [8]:
# I ended up using this a lot in the end
def get_table_data_from_html(soup) -> list:
    '''
    Extract tables from bs4 object and return a list of dataframes.
    '''
    
    # get all tables in the html
    tables = soup.findAll('table')

    # create dfs for each table and append each one to a list
    dfs_from_tables = []
    for table in tables:
        dfs_from_tables.append(pd.read_html(StringIO(str(table)))[0])
    
    return dfs_from_tables

In [134]:
def get_all_player_match_data(excel_file_path) -> tuple[pd.DataFrame, list]:
    '''
    Input an excel file of the copied tables from the websites and return a tuple
    where the first element is a dataframe of all player data and the second element
    is a list of URLs where extraction failed. This attempts to extract data from
    the failed URLs once more before completing.
    '''
    # get year from directory the excel file is in
    year = os.path.dirname(excel_file_path).split('\\')[-1].replace('mls_', '')
    # extract all links from the excel file
    # returned as a df in case of future use with an idea I had
    player_links = get_all_player_match_data_links(excel_file_path, year)
    # initialize empty df for all player data
    player_data_df = pd.DataFrame()
    
    # generate the initial player data df and failed links list
    player_data_df, failed_links = generate_player_df(list(player_links['stat_link']), player_data_df)
    
    # if first round has failed links, retry them
    # sometimes links failed for some reason but rerunning them seemed to fix it
    if len(failed_links)>0:
        player_data_df, failed_links = generate_player_df(failed_links, player_data_df)

    return player_data_df, failed_links 

def generate_player_df(player_links, df=pd.DataFrame()) -> tuple[pd.DataFrame, list]:
    '''
    Given an iterable of individual player links (and a df to modify), return a tuple (DataFrame, list)
    where the dataframe contains all player data and the list contains URLs with failed extractions.
    '''
    # get total number of players for alive_bar
    total_players = len(player_links)
    # to append failed links to a list
    failed_links = []
    
    # progress bar, force_tty=True might be needed depending if animations don't show for you
    # I used jupyter notebooks, so I needed this
    with alive_bar(total_players, force_tty=True) as bar:
        for player_url in player_links:
            # limited to 10 requests a minute per website rules :(
            # I set this as 7 seconds just to be safe but you can adjust as needed
            # in the config file
            time.sleep(int(config['other']['request_time_limit']))
            
            # if you get a player df, append it to df, otherwise add to failed links
            temp_df = attempt_data_extraction(player_url)
            if type(temp_df)==type(None):
                failed_links.append(player_url)
            else:
                df = pd.concat([df, temp_df], ignore_index=True)
            bar()
    # return df and failed links
    return df, failed_links

def attempt_data_extraction(url) -> pd.DataFrame | None:
    '''
    Attempt to get a player data dataframe. Return the df if successful,
    otherwise return None.
    '''
    # try to get a player_df and return it
    try:
        player_df = get_player_data_df(url)
        return player_df
    # otherwise, let the user know and return nothing
    except:
        print(f"Could not get player data for {url}")
        return None

def get_all_player_match_data_links(excel_file_path, year) -> pd.DataFrame:
    '''
    Extract all links from the excel that is a copy of the website data. Return
    a dataframe of the player names and URLs.
    '''
    # read excel file
    all_players = pd.read_excel(excel_file_path)
    wb = openpyxl.load_workbook(excel_file_path)
    sheets = wb.sheetnames
    ws = wb[sheets[0]]
    # get hyper links from file
    # in this situation, links didn't start until +2 and are found in column 37
    # this may need to be adjusted depending on how the data comes out
    all_players['stat_link'] = [ws.cell(row=i+2, column=37).hyperlink.target for i in range(all_players.shape[0])]
    # save all links out
    all_players[['Player', 'stat_link']].to_csv(output/f'mls_{year}' / 'player_links.csv', index=False)
    
    # return df of just the player name and stat link
    return all_players[['Player', 'stat_link']]
    
    
def get_player_data_df(url) -> pd.DataFrame:
    '''
    Using the URL for the individual player, get the first table on the website
    and return it as a dataframe.
    '''
    # get html data from url
    player_data_html = get_html_data(url)
    # generate tables list
    player_df = get_table_data_from_html(player_data_html)
    # add player url to data in first df in list
    # the lists always had 1 df but would break unless I left it as-is
    player_df[0]['player'] = player_url
    
    return player_df[0]

# Current Season - 2024

In [5]:
# this url gives me a list of all players in the current league
base_url = 'https://fbref.com/en/comps/22/Major-League-Soccer-Stats'

# this page gives me a bunch of tables for team stats in the current moment
# not sure how much of the data here will be useful, but I'll grab it just-in-case
html = get_html_data(base_url + 'players/')

In [6]:
team_stat_dfs = get_table_data_from_html(html)

In [7]:
# table names in the html
tables = ['eastern_conference',
          'western_conference',
          'squad_standard_stats',
          'squad_goalkeeping',
          'squad_advanced_goalkeeping',
          'squad_shooting',
          'squad_passing',
          'squad_pass_types',
          'squad_goal_and_shot_creation',
          'squad_defensive_actions',
          'squad_possession',
          'squad_playing_time',
          'squad_miscellaneous_stats']

# table pairings based on index in the list
pairs = list(zip([i for i in range(0,26,2)], [i for i in range(1,27,2)]))

# create csv files for each table
for index, pair in enumerate(pairs):
    pair_df = pd.concat([team_stat_dfs[pair[0]], team_stat_dfs[pair[1]]],axis=1)
    pair_df.to_csv(team_stats/f'{tables[index]}.csv',index=False)

# Individual Player Data
I had to copy and paste the "Player Standard Stats" table from this url (https://fbref.com/en/comps/22/stats/Major-League-Soccer-Stats) since I failed to do so with bs4. I saved it
out as an excel file. I just want to extract the urls that go directly to the players stats.

In [ ]:
player_data_2022_df, failed_links_2024 = get_all_player_match_data(config['paths']['all_players_2023'])

In [ ]:
player_data_2024_df.to_csv(output/"mls_2024"/"all_player_data_2024.csv")

# PRIOR YEARS
I recommend running each year/season individually. At the time of creating this script,
the website has a limit of 10 requests/minute. As a result, I had to stall with a timer
of 7 seconds before running each request. This causes each year to take ~2 hours just to
get the individual player performance data for each match. It would be best to finish
tackling a year before moving onto the next. Depending on your PC, you might also need
to take resources into account as well and save each year out before moving onto the next.

# 2023 Season

In [ ]:
player_data_2022_df, failed_links_2023 = get_all_player_match_data(config['paths']['all_players_2023'])

In [ ]:
player_data_2023_df.to_csv(output/"mls_2023"/"all_player_data_2023.csv")

# 2022 Season

In [135]:
player_data_2022_df, failed_links_2022 = get_all_player_match_data(config['paths']['all_players_2022'])

on 129: Could not get player data for https://fbref.com/en/players/1a160ea3/matchlogs/2022/summary/Geoff-Cameron-Match-Logs
|████████████████████████████████████████| 784/784 [100%] in 1:43:27.9 (0.13/s) 
|████████████████████████████████████████| 1/1 [100%] in 7.8s (0.13/s)          


In [140]:
player_data_2022_df.to_csv(output/"mls_2022"/"all_player_data_2022.csv")

# Sorting More
Work in progress

In [27]:
df_2022 = pd.read_csv(config['paths']['all_player_data_2022'])
# df_2023 = pd.read_csv(config['paths']['all_player_data_2023'])

In [28]:
df_2022

,Unnamed: 0,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,...,Passes,Passes.1,Passes.2,Passes.3,Carries,Carries.1,Take-Ons,Take-Ons.1,Unnamed: 36_level_0,player
0,NaN,Date,Day,Comp,Round,Venue,Result,Squad,Opponent,Start,...,Cmp,Att,Cmp%,PrgP,Carries,PrgC,Att,Succ,Match Report,NaN
1,0.0,2022-02-26,Sat,MLS,Regular Season,Home,D 1–1,Philadelphia Union,Minnesota Utd,N,...,6,7,85.7,2,6,1,0,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...
2,1.0,2022-03-05,Sat,MLS,Regular Season,Away,W 2–1,Philadelphia Union,CF Montréal,N,...,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",Match Report,https://fbref.com/en/players/672f4300/matchlog...
3,2.0,2022-03-12,Sat,MLS,Regular Season,Home,W 2–0,Philadelphia Union,SJ Earthquakes,N,...,4,4,100.0,1,7,1,1,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...
4,3.0,2022-03-19,Sat,MLS,Regular Season,Away,W 2–0,Philadelphia Union,NYCFC,N,...,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",Match Report,https://fbref.com/en/players/672f4300/matchlog...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26667,26666.0,2022-10-01,Sat,MLS,Regular Season,Home,L 2–3,FC Cincinnati,Chicago Fire,Y,...,30,38,78.9,4,16,0,1,1,Match Report,https://fbref.com/en/players/672f4300/matchlog...
26668,26667.0,2022-10-09,Sun,MLS,Regular Season,Away,W 5–2,FC Cincinnati,D.C. United,Y,...,16,21,76.2,2,15,0,0,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...
26669,26668.0,2022-10-15,Sat,MLS,First Round,Away,W 2–1,FC Cincinnati,NY Red Bulls,Y,...,10,15,66.7,1,5,0,0,0,Match Report,https://fbref.com/en/players/672f4300/matchlog...
26670,26669.0,2022-10-20,Thu,MLS,Conference Semifinals,Away,L 0–1,FC Cincinnati,Philadelphia Union,N,...,"On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play","On matchday squad, but did not play",Match Report,https://fbref.com/en/players/672f4300/matchlog...


In [204]:
df_2022 = df_2022[df_2022['Unnamed: 2_level_0']=="MLS"]

In [205]:
game_schedule_df = df_2022[['Unnamed: 0_level_0', 'Unnamed: 1_level_0','Unnamed: 2_level_0', 'Unnamed: 3_level_0', 'Unnamed: 4_level_0', 'Unnamed: 6_level_0','Unnamed: 7_level_0', 'Unnamed: 5_level_0']]

In [206]:
game_schedule_df

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 5_level_0
1,2022-02-26,Sat,MLS,Regular Season,Home,Philadelphia Union,Minnesota Utd,D 1–1
2,2022-03-05,Sat,MLS,Regular Season,Away,Philadelphia Union,CF Montréal,W 2–1
3,2022-03-12,Sat,MLS,Regular Season,Home,Philadelphia Union,SJ Earthquakes,W 2–0
4,2022-03-19,Sat,MLS,Regular Season,Away,Philadelphia Union,NYCFC,W 2–0
5,2022-04-02,Sat,MLS,Regular Season,Home,Philadelphia Union,Charlotte,W 2–0
...,...,...,...,...,...,...,...,...
26666,2022-09-27,Tue,MLS,Regular Season,Away,FC Cincinnati,Seattle Sounders FC,D 1–1
26667,2022-10-01,Sat,MLS,Regular Season,Home,FC Cincinnati,Chicago Fire,L 2–3
26668,2022-10-09,Sun,MLS,Regular Season,Away,FC Cincinnati,D.C. United,W 5–2
26669,2022-10-15,Sat,MLS,First Round,Away,FC Cincinnati,NY Red Bulls,W 2–1


In [207]:
game_schedule_df = game_schedule_df[game_schedule_df['Unnamed: 4_level_0']=='Home']

In [210]:
game_schedule_df['result'] = game_schedule_df['Unnamed: 5_level_0'].apply(lambda x: x.replace(' ', '–').replace('-','–').split('–'))

C:\Users\Cristian\AppData\Local\Temp\ipykernel_27384\3937752280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_schedule_df['result'] = game_schedule_df['Unnamed: 5_level_0'].apply(lambda x: x.replace(' ', '–').replace('-','–').split('–'))


In [211]:
game_schedule_df

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 5_level_0,result
1,2022-02-26,Sat,MLS,Regular Season,Home,Philadelphia Union,Minnesota Utd,D 1–1,"[D, 1, 1]"
3,2022-03-12,Sat,MLS,Regular Season,Home,Philadelphia Union,SJ Earthquakes,W 2–0,"[W, 2, 0]"
5,2022-04-02,Sat,MLS,Regular Season,Home,Philadelphia Union,Charlotte,W 2–0,"[W, 2, 0]"
6,2022-04-09,Sat,MLS,Regular Season,Home,Philadelphia Union,Columbus Crew,W 1–0,"[W, 1, 0]"
8,2022-04-23,Sat,MLS,Regular Season,Home,Philadelphia Union,CF Montréal,D 1–1,"[D, 1, 1]"
...,...,...,...,...,...,...,...,...,...
26659,2022-08-13,Sat,MLS,Regular Season,Home,FC Cincinnati,Atlanta Utd,D 2–2,"[D, 2, 2]"
26661,2022-08-27,Sat,MLS,Regular Season,Home,FC Cincinnati,Columbus Crew,D 2–2,"[D, 2, 2]"
26662,2022-09-03,Sat,MLS,Regular Season,Home,FC Cincinnati,Charlotte,W 2–0,"[W, 2, 0]"
26664,2022-09-10,Sat,MLS,Regular Season,Home,FC Cincinnati,SJ Earthquakes,W 6–0,"[W, 6, 0]"


In [1]:
game_schedule_df['home_score'] = game_schedule_df['result'].apply(lambda x: int(x[1]))
game_schedule_df['away_score'] = game_schedule_df['result'].apply(lambda x: int(x[2]) if len(x)==5 else int(x[5]))
game_schedule_df['result'] = game_schedule_df['result'].apply(lambda x: x[0])
game_schedule_df.drop_duplicates(inplace=True)
game_schedule_df['season'] = 2022

In [215]:
game_schedule_df[game_schedule_df['Unnamed: 0_level_0']=='2022-10-17']

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 5_level_0,result,home_score
110,2022-10-17,Mon,MLS,First Round,Home,NYCFC,Inter Miami,W 3–0,"[W, 3, 0]",3
401,2022-10-17,Mon,MLS,First Round,Home,NYCFC,Inter Miami,W 3–0,"[W, 3, 0]",3
1003,2022-10-17,Mon,MLS,First Round,Home,NYCFC,Inter Miami,W 3–0,"[W, 3, 0]",3
1578,2022-10-17,Mon,MLS,First Round,Home,FC Dallas,Minnesota Utd,D 1 (5)–1 (4),"[D, 1, (5), 1, (4)]",1
4116,2022-10-17,Mon,MLS,First Round,Home,NYCFC,Inter Miami,W 3–0,"[W, 3, 0]",3
4884,2022-10-17,Mon,MLS,First Round,Home,FC Dallas,Minnesota Utd,D 1 (5)–1 (4),"[D, 1, (5), 1, (4)]",1
4971,2022-10-17,Mon,MLS,First Round,Home,NYCFC,Inter Miami,W 3–0,"[W, 3, 0]",3
7769,2022-10-17,Mon,MLS,First Round,Home,FC Dallas,Minnesota Utd,D 1 (5)–1 (4),"[D, 1, (5), 1, (4)]",1
7951,2022-10-17,Mon,MLS,First Round,Home,FC Dallas,Minnesota Utd,D 1 (5)–1 (4),"[D, 1, (5), 1, (4)]",1
9338,2022-10-17,Mon,MLS,First Round,Home,NYCFC,Inter Miami,W 3–0,"[W, 3, 0]",3


In [124]:
game_schedule_df

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 5_level_0,result,home_score,away_score,season
1,2022-02-26,Sat,MLS,Regular Season,Home,Philadelphia Union,Minnesota Utd,D 1–1,D,1,1,2022
3,2022-03-12,Sat,MLS,Regular Season,Home,Philadelphia Union,SJ Earthquakes,W 2–0,W,2,0,2022
5,2022-04-02,Sat,MLS,Regular Season,Home,Philadelphia Union,Charlotte,W 2–0,W,2,0,2022
6,2022-04-09,Sat,MLS,Regular Season,Home,Philadelphia Union,Columbus Crew,W 1–0,W,1,0,2022
8,2022-04-23,Sat,MLS,Regular Season,Home,Philadelphia Union,CF Montréal,D 1–1,D,1,1,2022
...,...,...,...,...,...,...,...,...,...,...,...,...
6280,2022-05-28,Sat,MLS,Regular Season,Home,Sporting KC,Vancouver W'caps,L 0–1,L,0,1,2022
6285,2022-07-03,Sun,MLS,Regular Season,Home,Sporting KC,NY Red Bulls,L 0–1,L,0,1,2022
6922,2022-04-09,Sat,MLS,Regular Season,Home,Sporting KC,Nashville SC,L 1–2,L,1,2,2022
6924,2022-04-23,Sat,MLS,Regular Season,Home,Sporting KC,Columbus Crew,D 0–0,D,0,0,2022


In [125]:
game_schedule_df.columns = ['game_date', 'day', 'league', 'round', 'ishome', 'home_team', 'away_team','result_uncleaned', 'result', 'home_score', 'away_score', 'season_year']
game_schedule_df = game_schedule_df[['game_date', 'day', 'league', 'round', 'home_team', 'away_team', 'result', 'home_score', 'away_score', 'season_year']]

In [213]:
game_schedule_df.to_csv(output / 'mls_2022' /'match_schedule.csv')

In [190]:
game_schedule_df['home_score'][1]

'1'

In [127]:
game_schedule_df['game_date'] = game_schedule_df['game_date'].apply(lambda x: )

,game_date,day,league,round,home_team,away_team,result,home_score,away_score,season_year
1,2022-02-26,Sat,MLS,Regular Season,Philadelphia Union,Minnesota Utd,D,1,1,2022
3,2022-03-12,Sat,MLS,Regular Season,Philadelphia Union,SJ Earthquakes,W,2,0,2022
5,2022-04-02,Sat,MLS,Regular Season,Philadelphia Union,Charlotte,W,2,0,2022
6,2022-04-09,Sat,MLS,Regular Season,Philadelphia Union,Columbus Crew,W,1,0,2022
8,2022-04-23,Sat,MLS,Regular Season,Philadelphia Union,CF Montréal,D,1,1,2022
...,...,...,...,...,...,...,...,...,...,...
6280,2022-05-28,Sat,MLS,Regular Season,Sporting KC,Vancouver W'caps,L,0,1,2022
6285,2022-07-03,Sun,MLS,Regular Season,Sporting KC,NY Red Bulls,L,0,1,2022
6922,2022-04-09,Sat,MLS,Regular Season,Sporting KC,Nashville SC,L,1,2,2022
6924,2022-04-23,Sat,MLS,Regular Season,Sporting KC,Columbus Crew,D,0,0,2022


In [182]:
import sqlite3

In [186]:
conn = sqlite3.connect('../src/mls.db')
game_schedule_df.to_sql('match_schedules', conn)

489

In [ ]:
data_injection = '''
insert into table_name(game_date, day, league, round, home_team, away_team, result, home_score, away_score, season_year)
values {}
'''